Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip install -r requirements.txt

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 736, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 736 (delta 10), reused 20 (delta 8), pack-reused 713
Receiving objects: 100% (736/736), 348.62 MiB | 24.08 MiB/s, done.
Resolving deltas: 100% (161/161), done.
/content/tortoise-tts
     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 31.3 MB/s 
     |████████████████████████████████| 235 kB 44.3 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 38.9 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=bb7d90605d0bf4d89aedc46bd8ed39538f55e00ee70fa382c1af81f142f08fa8
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
  Created wheel for entmax: 

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

from api import TextToSpeech
from utils.audio import load_audio, get_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

Done.


Done.


Done.


Done.


Done.
Removing weight norm...


In [ ]:
# List all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls voices

angelina_jolie/  halle_barry/        lj/               samuel_jackson/
atkins/          harris/             mol/              sigourney_weaver/
carlin/          henry_cavill/       morgan_freeman/   tom_hanks/
daniel_craig/    jennifer_lawrence/  myself/           william_shatner/
dotrice/         john_krasinski/     otto/
emma_stone/      kennard/            patrick_stewart/
grace/           lescault/           robert_deniro/


In [ ]:
# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick one of the voices from above
voice = 'dotrice'
# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [ ]:
# Fetch the voice references and forward execute!
voices = get_voices()
cond_paths = voices[voice]
conds = []
for cond_path in cond_paths:
    c = load_audio(cond_path, 22050)
    conds.append(c)

gen = tts.tts_with_preset(text, conds, preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)

100%|██████████| 6/6 [01:18<00:00, 13.11s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/content/tortoise-tts/models/autoregressive.py:359: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  mel_lengths = wav_lengths // self.mel_length_compression


Performing vocoding..


100%|██████████| 32/32 [00:16<00:00,  1.94it/s]


In [ ]:
# You can add as many conditioning voices as you want together. Combining
# clips from multiple voices takes the mean of the latent space for all
# voices. This creates a novel voice that is a combination of the two inputs.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
conds = []
for v in ['patrick_stewart', 'william_shatner']:
  cond_paths = voices[v]
  for cond_path in cond_paths:
      c = load_audio(cond_path, 22050)
      conds.append(c)

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", conds, preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)

100%|██████████| 6/6 [01:45<00:00, 17.62s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/content/tortoise-tts/models/autoregressive.py:359: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  mel_lengths = wav_lengths // self.mel_length_compression


Performing vocoding..


100%|██████████| 32/32 [00:16<00:00,  2.00it/s]
